In [1]:
# %xmode Minimal

from qadence2_expressions import *

## Basic symbol examples

In [2]:
a = parameter('a')
b = parameter('b')

exprs = [
    "a + a",
    "a - a",
    "a / a",
    "a + b",
    "a / (2*b)",
    "a ** 0",
    "a ** 1",
    "2 ** (a + b)",
]

for expr in exprs:
    print(expr, "->", eval(expr))

a + a -> 2.0 a
a - a -> 0.0
a / a -> 1.0
a + b -> a + b
a / (2*b) -> a (2.0 b) ^ -1.0
a ** 0 -> 1.0
a ** 1 -> a
2 ** (a + b) -> 2.0 ^ (a + b)


Product of sums is automatically expanded.

In [3]:
(a + b) * (a + b)

a ^ 2.0 + 2.0 a b + b ^ 2.0

However, power of sums are not automatically expanded to prioritise simplifications with respect to power.

In [4]:
(a + b) * (a + b) ** 2 / (a + b)

(a + b) ^ 2.0

## Quantum operators

In [5]:
X(2) * Y() * Z(1,2)

X[2] Y[*] Z[1 2]

In [6]:
# Simple Hermitian operators
CZ = unitary_hermitian_operator('CZ')
NOT = unitary_hermitian_operator('NOT')

In [7]:
NOT(control=(1,), target=(2,)) * NOT(target=(1,), control=(2,))

NOT[2; 1] NOT[1; 2]

In [8]:
X() * (cos(a) * X() + 1j * sin(a) * Y()) / 2

0.5 cos(a) + 0.5j sin(a) X[*] Y[*]

In [9]:
X(1) * (cos(a) * X() + 1j * sin(a) * Y()) * Z(1) / 2

0.5 cos(a) X[1] X[*] Z[1] + 0.5j sin(a) X[1] Y[*] Z[1]

In [10]:
CNOT = lambda ctrl, tgt: NOT(target=(tgt,), control=(ctrl,))

Y(4) * X(3) * Y(5,4) * CNOT(1,2) * Z(3)

NOT[2; 1] X[3] Z[3] Y[4] Y[4 5]

In [11]:
n = lambda k: (1 - Z(k)) / 2

sum(a * X(i) - b * Z(i) + n(i) * n(i + 1) for i in range(1))

0.25 + a X[0] - b Z[0] - 0.25 Z[1] - 0.25 Z[0] + 0.25 Z[0] Z[1]

In [12]:
phi = variable("phi")

RX(phi / 2)(1) * RX(- phi / 2)(2)

RX(0.5 phi)[1] RX(-0.5 phi)[2]

In [13]:
RX(phi / 2)(1) * RX(- phi / 2)(1)

1.0

## IR compilation

In [14]:
expr = RX(a * phi / 2)(2) * CZ() * RY(b * phi / 2)(0)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=3,
    qubit_positions=[],
    grid_type=None,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    'a': Alloc(1, trainable=False),
    'phi': Alloc(1, trainable=True),
    'b': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('a'))),
    Assign('%1', Call('mul', Load('%0'), Load('phi'))),
    QuInstruct('rx', Support(target=(2,)), Load('%1')),
    QuInstruct('cz', Support.target_all()),
    Assign('%2', Call('mul', 0.5, Load('b'))),
    Assign('%3', Call('mul', Load('%2'), Load('phi'))),
    QuInstruct('ry', Support(target=(0,)), Load('%3')),
  ],
  directives={},
  settings={},
)

In [15]:
set_qubits_positions([(-2, 1), (0, 0), (3, 1)])
irc(expr)

Model(
  register=AllocQubits(
    num_qubits=3,
    qubit_positions=[(-2, 1), (0, 0), (3, 1)],
    grid_type=None,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    'a': Alloc(1, trainable=False),
    'phi': Alloc(1, trainable=True),
    'b': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('a'))),
    Assign('%1', Call('mul', Load('%0'), Load('phi'))),
    QuInstruct('rx', Support(target=(2,)), Load('%1')),
    QuInstruct('cz', Support.target_all()),
    Assign('%2', Call('mul', 0.5, Load('b'))),
    Assign('%3', Call('mul', Load('%2'), Load('phi'))),
    QuInstruct('ry', Support(target=(0,)), Load('%3')),
  ],
  directives={},
  settings={},
)

In [16]:
set_qubits_positions([(-1,0), (-1, 1), (1, 0), (1, 1)])
set_grid_type("triangular")

t = time_variable("t")
omega = array_variable("omega", 4)
detuning = array_variable("detuning", 3)
phase = parameter("phase")

expr = (
    NativeDriven(t / 2, omega, detuning, phase)()
    * FreeEvolution(2.5)()
    * NativeDriven(t / 2, omega, -detuning, phase)()
)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=4,
    qubit_positions=[(-1, 0), (-1, 1), (1, 0), (1, 1)],
    grid_type=triangular,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    't': Alloc(1, trainable=True),
    'omega': Alloc(4, trainable=True),
    'detuning': Alloc(3, trainable=True),
    'phase': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('t'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('detuning'), Load('phase')),
    QuInstruct('dyn_wait', Support.target_all(), 2.5),
    Assign('%1', Call('mul', -1.0, Load('detuning'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('%1'), Load('phase')),
  ],
  directives={},
  settings={},
)